# Lab 02: Handwritten Digit Recognition App

**Group 09** 

---

## 1. Introduction
This notebook serves as the final application interface. It allows users to:
1.  **Upload** an image of a handwritten digit (or use sample images).
2.  **Process** the image (grayscale, resize, feature extraction).
3.  **Predict** the digit using our best trained Softmax Regression model.
4.  **Visualize** the prediction confidence.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from io import BytesIO
from PIL import Image

# --- SOFTMAX REGRESSION CLASS (Inference Only) ---
class SoftmaxRegression:
    def __init__(self):
        self.W = None
        self.b = None

    def softmax(self, z):
        z_stable = z - np.max(z, axis=1, keepdims=True)
        exp_z = np.exp(z_stable)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def predict_proba(self, X):
        z = np.dot(X, self.W) + self.b
        return self.softmax(z)

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

    def load_weights(self, filepath):
        if not os.path.exists(filepath):
            print(f"❌ Error: Weight file not found at {filepath}")
            return False
        
        try:
            data = np.load(filepath)
            self.W = data['W']
            self.b = data['b']
            print(f"✅ Model loaded successfully from {filepath}")
            print(f"   - Weights Shape: {self.W.shape}")
            print(f"   - Bias Shape: {self.b.shape}")
            return True
        except Exception as e:
            print(f"❌ Error loading weights: {e}")
            return False

# Initialize and Load
model = SoftmaxRegression()
weight_path = "../models/best_model_weights.npz"
model_ready = model.load_weights(weight_path)

✅ Model loaded successfully from ../models/best_model_weights.npz
   - Weights Shape: (81, 10)
   - Bias Shape: (1, 10)


## 2. Smart Feature Pipeline

Since we experimented with different features (Raw, Deskewed, HOG), the application must define which processing pipeline to use based on the loaded model's input dimension ($D$).

* **If $D = 784$:** The model uses Pixels. We will apply **Deskewing** + **Normalization** to give it the best chance (as Deskewing improves raw pixel models).
* **If $D = 81$:** The model uses **HOG**. We will compute HOG features.
* **Other:** Fallback to raw resizing (or error if PCA was used without saved components).

In [2]:
# --- 1. DESKEWING FUNCTION ---
def deskew_image(img):
    """Straightens the digit."""
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2: return img.copy()
    skew = m['mu11'] / m['mu02']
    M = np.float32([[1, skew, -0.5*28*skew], [0, 1, 0]])
    return cv2.warpAffine(img, M, (28, 28), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)

# --- 2. HOG FUNCTION ---
def extract_hog_feature(img):
    """Extracts 81 HOG features from a single image."""
    # Ensure params match training
    hog = cv2.HOGDescriptor(_winSize=(28, 28), _blockSize=(14, 14), _blockStride=(7, 7), _cellSize=(14, 14), _nbins=9)
    # HOG needs uint8
    img_uint8 = (img * 255).astype(np.uint8) if img.max() <= 1.0 else img.astype(np.uint8)
    return hog.compute(img_uint8).flatten()

# --- 3. MASTER PREPROCESSOR ---
def process_input(image_pil, model_input_dim):
    """
    Converts PIL image to model input vector based on dimension.
    """
    # A. Basic Processing (Grayscale -> Resize -> Invert -> Numpy)
    img = image_pil.convert('L')
    img = img.resize((28, 28), Image.Resampling.LANCZOS)
    img_arr = np.array(img)
    
    # Invert if background is white (common in drawing apps/paper)
    if np.mean(img_arr) > 128:
        img_arr = 255 - img_arr
    
    # Normalize to 0-1 float for deskewing/raw
    img_norm = img_arr.astype(np.float32) / 255.0
    
    # B. Branch based on Model Dimension
    
    # Case 1: HOG Model (81 features)
    if model_input_dim == 81:
        # HOG handles its own normalization, pass the deskewed version for better results?
        # Let's just pass the basic one to match training pipeline usually.
        # But deskewing helps HOG too. Let's apply Deskew -> HOG
        img_deskew = deskew_image((img_norm * 255).astype(np.uint8))
        features = extract_hog_feature(img_deskew)
        return img_arr, img_deskew, features.reshape(1, -1), "HOG (81)"
        
    # Case 2: Pixel Model (784 features)
    elif model_input_dim == 784:
        # Always Apply Deskewing for best performance on 784 models
        img_deskew = deskew_image((img_norm * 255).astype(np.uint8))
        img_final = img_deskew.astype(np.float32) / 255.0
        return img_arr, img_deskew, img_final.reshape(1, -1), "Deskewed Pixels (784)"
    
    # Case 3: PCA or others (Not supported in simple demo without saving components)
    else:
        return img_arr, img_arr, None, f"Unsupported Dim: {model_input_dim}"

In [3]:
# --- WIDGET UI ---

# 1. Widgets
uploader = widgets.FileUpload(accept='image/*', multiple=False, description='Upload Digit')
btn_predict = widgets.Button(description='Predict', button_style='primary', icon='magic')
out_display = widgets.Output()

def on_click_predict(b):
    out_display.clear_output()
    
    if not model_ready:
        with out_display: print("❌ Model not loaded. Run Notebook 2 first.")
        return
        
    if not uploader.value:
        with out_display: print("⚠️ Please upload an image first.")
        return

    # Load Image
    try:
        # Handle different ipywidgets versions
        file_info = uploader.value[0] if isinstance(uploader.value, tuple) else uploader.value
        # Access 'content' key safely
        if isinstance(file_info, dict):
            content = file_info.get('content')
            if content is None: # Sometimes it's nested differently
                 content = list(file_info.values())[0]['content']
        else: # List of dicts
             content = file_info[0]['content']
             
        img_pil = Image.open(BytesIO(content))
    except Exception as e:
        with out_display: print(f"Error reading file: {e}")
        return

    # Process
    input_dim = model.W.shape[0]
    img_orig, img_processed, X_input, method_name = process_input(img_pil, input_dim)
    
    if X_input is None:
        with out_display: print(f"❌ Error: {method_name}. Need PCA components file to run PCA models.")
        return

    # Predict
    probs = model.predict_proba(X_input)[0]
    pred_label = np.argmax(probs)
    confidence = probs[pred_label]

    # Visualize
    with out_display:
        fig, ax = plt.subplots(1, 3, figsize=(12, 4))
        
        # 1. Original
        ax[0].imshow(img_orig, cmap='gray')
        ax[0].set_title("Original Input")
        ax[0].axis('off')
        
        # 2. Processed (What model sees)
        ax[1].imshow(img_processed, cmap='gray')
        ax[1].set_title(f"Processed: {method_name}")
        ax[1].axis('off')
        
        # 3. Bar Chart
        bars = ax[2].bar(range(10), probs, color='skyblue')
        bars[pred_label].set_color('orange')
        ax[2].set_xticks(range(10))
        ax[2].set_title(f"Prediction: {pred_label} ({confidence:.1%})")
        ax[2].set_ylim(0, 1)
        
        plt.tight_layout()
        plt.show()

btn_predict.on_click(on_click_predict)

# Layout
display(widgets.VBox([
    widgets.HTML("<h3>✍️ Handwritten Digit Recognizer</h3>"),
    widgets.HBox([uploader, btn_predict]),
    out_display
]))